Imports

In [55]:
from os import getenv
import json

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlalchemy as db
from dotenv import load_dotenv

load_dotenv()

image_path = 'C:/Users/tabm9/OneDrive - Caissa Analytica/Documents/DS_Portfolio/Projects/Images/RLCS'

Connect to Database

In [9]:
# DB connection
cnx = db.create_engine(f"postgresql://{getenv('USER')}:{getenv('PASS')}@{getenv('HOST')}:{getenv('PORT')}/{getenv('DB')}")

# Elo Tests

In [10]:
# Auxiliary functions
def df_from_table(table_name: str):
    return pd.read_sql(f'select * from {table_name}', cnx)

def expected_score(rating_a, rating_b):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    expected_b = 1 / (1 + 10 ** ((rating_a - rating_b) / 400))
    return expected_a, expected_b

def K_days_update(K_lower, K_upper, players, game_date, d=30):
    D = sum([min(d, (game_date - players[item]).days) for item in players.keys()])
    return K_upper - (K_upper - K_lower) * D / (d * 3)

def K_games_update(K_lower, K_upper, players, g=50):
    G = sum([min(g, players[item]) for item in players.keys()])
    return K_upper - (K_upper - K_lower) * G / (g * 3)


In [11]:
# Extract game history
games_df = df_from_table('rocket_league.pre_elo_games')

## Naive Elo

In [22]:

def calculate_naive_elo(df: pd.DataFrame, k: int, rating_window: int = 10, diff_window: int = 20):
    current_ratings = {}
    ratings_history = []

    for _, game in df.iterrows():
        # Initialize entry if not exists
        if game['winner_team_id'] not in set(current_ratings.keys()):
            current_ratings[game['winner_team_id']] = {
                'date': game['date'],
                'rating': 2000,
            }
        
        if game['loser_team_id'] not in set(current_ratings.keys()):
            current_ratings[game['loser_team_id']] = {
                'date': game['date'],
                'rating': 2000,
            }

        # Update ratings
        rating_winner = current_ratings[game['winner_team_id']]['rating']
        rating_loser = current_ratings[game['loser_team_id']]['rating']

        expected_winner, expected_loser = expected_score(rating_winner, rating_loser)

        current_ratings[game['winner_team_id']]['rating'] = rating_winner + k * (1 - expected_winner)
        current_ratings[game['loser_team_id']]['rating'] = rating_loser + k * (0 - expected_loser)

        ratings_history.append({
            'date': game['date'],
            'team_id': game['winner_team_id'],
            'team_name': game['winner_team_name'],
            'roster_id': game['winner_roster_id'],
            'team_players': game['winner_team_players'],
            'rating': current_ratings[game['winner_team_id']]['rating'],
            'S': 1,
            'E': expected_winner,
            'min_D': min(expected_winner, 1 - expected_winner)
        })
        ratings_history.append({
            'date': game['date'],
            'team_id': game['loser_team_id'],
            'team_name': game['loser_team_name'],
            'roster_id': game['loser_roster_id'],
            'team_players': game['loser_team_players'],
            'rating': current_ratings[game['loser_team_id']]['rating'],
            'S': 0,
            'E': expected_loser,
            'min_D': min(expected_loser, 1 - expected_loser)
        })

    ratings_history = pd.DataFrame(ratings_history)
    ratings_history['D'] = ratings_history['S'] - ratings_history['E']
    ratings_history['D_u'] = abs(ratings_history['D']) - ratings_history['min_D']

    return ratings_history, current_ratings

ratings_history, current_ratings = calculate_naive_elo(games_df, 8)
ratings_history

,date,team_id,team_name,roster_id,team_players,rating,S,E,min_D,D,D_u
0,2015-09-08 00:00:00,6020be74f1e4807cc7013fe8,Perfect Storm,0.0,"5f3d8fdd95f40596eae23d7b,5f3d8fdd95f40596eae23...",2004.000000,1,0.500000,0.500000,0.500000,0.000000
1,2015-09-08 00:00:00,6020bc70f1e4807cc70023c7,FlipSid3 Tactics,0.0,"5f3d8fdd95f40596eae23d97,5f3d8fdd95f40596eae23...",1996.000000,0,0.500000,0.500000,-0.500000,0.000000
2,2015-09-08 00:00:00,6020be74f1e4807cc7013f9b,Cosmic Aftershock,1.0,"5f3d8fdd95f40596eae23d7e,5f3d8fdd95f40596eae23...",2004.000000,1,0.500000,0.500000,0.500000,0.000000
3,2015-09-08 00:00:00,6020be74f1e4807cc7014004,Air Force Two,0.0,"5f3d8fdd95f40596eae23f2d,5f3d8fdd95f40596eae23...",1996.000000,0,0.500000,0.500000,-0.500000,0.000000
4,2015-09-08 00:00:00,6020bc70f1e4807cc70023c7,FlipSid3 Tactics,0.0,"5f3d8fdd95f40596eae23d97,5f3d8fdd95f40596eae23...",2000.092087,1,0.488489,0.488489,0.511511,0.023022
...,...,...,...,...,...,...,...,...,...,...,...
194639,2022-12-11 18:23:03,632f2d34da9d7ca1c7bb4686,Gen.G Esports,0.0,"5f3d8fdd95f40596eae240d3,5f3d8fdd95f40596eae24...",2246.819244,0,0.599032,0.400968,-0.599032,0.198065
194640,2022-12-11 18:30:54,632f2d34da9d7ca1c7bb4686,Gen.G Esports,0.0,"5f3d8fdd95f40596eae240d3,5f3d8fdd95f40596eae24...",2250.133559,1,0.585711,0.414289,0.414289,0.000000
194641,2022-12-11 18:30:54,62750f2fc437fde7e02d5926,Moist Esports,1.0,"5f3d8fdd95f40596eae23f83,5f3d8fdd95f40596eae23...",2183.353139,0,0.414289,0.414289,-0.414289,0.000000
194642,2022-12-11 18:38:53,632f2d34da9d7ca1c7bb4686,Gen.G Esports,0.0,"5f3d8fdd95f40596eae240d3,5f3d8fdd95f40596eae24...",2253.374051,1,0.594938,0.405062,0.405062,0.000000


In [53]:
Ks = list(range(1, 20))
team = 'NRG Esports'

fig = go.Figure(layout=dict(title=f'Naive Ratings for {team} with Different Ks'))
for K in Ks:
    ratings_history, current_ratings = calculate_naive_elo(games_df, K)
    team_df = ratings_history[ratings_history['team_name'] == team].copy().reset_index(drop=True)
    fig.add_trace(go.Scatter(x=team_df.index, y=team_df['rating'], opacity=0.5, name=f'K = {K}'))

fig.write_html(image_path + f'/Naive_Elo_{team}.html')
fig.show(renderer='browser')

In [60]:
ratings_history, current_ratings = calculate_naive_elo(games_df, 6)

# Isolate Team
team = 'NRG Esports'
team_df = ratings_history[ratings_history['team_name'] == team].copy().reset_index(drop=True)

# Calculate volatility
v1 = [np.nan] * len(team_df)
v2 = [np.nan] * len(team_df)
D = team_df['D'].to_numpy()
D_u = team_df['D_u'].to_numpy()
N = 10

for i in range(10, len(team_df)):
    v1[i] = np.sqrt(sum(D[i - N: i]**2) / N)
    v2[i] = np.sqrt(sum(D_u[i - N: i]**2) / N)

team_df['volatility_1'] = v1
team_df['volatility_2'] = v2

# Create Plot
fig = make_subplots(rows=2, cols=1)
rosters = team_df['roster_id'].unique()
colors = iter(px.colors.qualitative.Plotly)
for roster in rosters:
    color = colors.__next__()
    fig_df = team_df[team_df['roster_id'] == roster].copy()
    fig.add_trace(go.Scatter(x=fig_df.index, y=fig_df['rating'], line=dict(color=color), name=f'Roster {int(roster)}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=fig_df.index, y=fig_df['volatility_1'], line=dict(color=color), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=fig_df.index, y=fig_df['volatility_2'], line=dict(color=color), opacity=0.5, showlegend=False), row=2, col=1)

fig.write_html(image_path + f'/Naive_Elo_Volattility_{team}.html')
fig.show(renderer='browser')

team_df

,date,team_id,team_name,roster_id,team_players,rating,S,E,min_D,D,D_u,volatility_1,volatility_2
0,2016-09-24 17:23:00,6020bc70f1e4807cc70023a0,NRG Esports,0.0,"5f3d8fdd95f40596eae23d7a,5f3d8fdd95f40596eae23...",1996.972808,0,0.504532,0.495468,-0.504532,0.009064,NaN,NaN
1,2016-09-24 17:32:00,6020bc70f1e4807cc70023a0,NRG Esports,0.0,"5f3d8fdd95f40596eae23d7a,5f3d8fdd95f40596eae23...",1999.997893,1,0.495819,0.495819,0.504181,0.008362,NaN,NaN
2,2016-09-24 17:41:00,6020bc70f1e4807cc70023a0,NRG Esports,0.0,"5f3d8fdd95f40596eae23d7a,5f3d8fdd95f40596eae23...",1996.970738,0,0.504526,0.495474,-0.504526,0.009052,NaN,NaN
3,2016-09-24 17:48:00,6020bc70f1e4807cc70023a0,NRG Esports,0.0,"5f3d8fdd95f40596eae23d7a,5f3d8fdd95f40596eae23...",1999.995858,1,0.495813,0.495813,0.504187,0.008374,NaN,NaN
4,2016-09-24 17:55:00,6020bc70f1e4807cc70023a0,NRG Esports,0.0,"5f3d8fdd95f40596eae23d7a,5f3d8fdd95f40596eae23...",1996.968738,0,0.504520,0.495480,-0.504520,0.009040,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,2022-11-04 20:17:05,6020bc70f1e4807cc70023a0,NRG Esports,7.0,"5f3d8fdd95f40596eae23d6f,5f3d8fdd95f40596eae23...",2147.664243,0,0.385715,0.385715,-0.385715,0.000000,0.445005,0.085980
2394,2022-11-04 20:49:15,6020bc70f1e4807cc70023a0,NRG Esports,7.0,"5f3d8fdd95f40596eae23d6f,5f3d8fdd95f40596eae23...",2143.998222,0,0.611004,0.388996,-0.611004,0.222007,0.445515,0.085980
2395,2022-11-04 20:57:05,6020bc70f1e4807cc70023a0,NRG Esports,7.0,"5f3d8fdd95f40596eae23d6f,5f3d8fdd95f40596eae23...",2140.392664,0,0.600926,0.399074,-0.600926,0.201852,0.471018,0.111001
2396,2022-11-04 21:04:31,6020bc70f1e4807cc70023a0,NRG Esports,7.0,"5f3d8fdd95f40596eae23d6f,5f3d8fdd95f40596eae23...",2142.847079,1,0.590931,0.409069,0.409069,0.000000,0.494425,0.128045
